In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
path_test = '/content/drive/MyDrive/ResNet Data/ResNet Data'
category = ['Cats', 'Dogs']

In [ ]:
training = []
def createTrainingData():
    for cate in category:
        path = os.path.join(path_test, cate)
        class_num = category.index(cate)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img))
            # gray_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
            new_array = cv2.resize(img_array, (64, 64))
            training.append([new_array, class_num])
createTrainingData()

In [ ]:
x = []
y = []
for img, cl in training:
  x.append(img)
  y.append(cl)

In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
type(x), type(y)

(list, list)

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
type(x), type(y)

(numpy.ndarray, numpy.ndarray)

In [ ]:
x.shape, y.shape

((960, 64, 64, 3), (960,))

In [ ]:
#y

In [ ]:
from keras.utils import np_utils
Y = np_utils.to_categorical(y, 2)
print(Y.shape)

(960, 2)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size=0.3, random_state=None)

In [ ]:
y_train

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, Flatten, MaxPool2D, Dropout, RandomFlip, RandomRotation
from tensorflow.keras.models import Sequential
from tensorflow import keras

In [ ]:
mobile = MobileNet(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
model = Sequential()
model.add(mobile)

# model.add(Conv2D(64, (3,3), activation = 'relu'))
# model.add(MaxPool2D((2,2), (1,1), 'same'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Conv2D(32 , (3,3), activation = 'relu'))
# model.add(MaxPool2D((2,2), (1,1), 'same'))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
model.add(RandomFlip("horizontal_and_vertical"))
model.add(RandomRotation(0.2))

model.add(Flatten())
model.add(Dense(16, activation= 'relu'))
model.add(Dropout(0.4))

model.add(Dense(8, activation= 'relu'))
model.add(Dropout(0.3))

model.add(Dense(2, activation = 'sigmoid'))

model.summary()

17235968/17225924 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 2, 2, 1024)       3228864   
 nal)                                                            
                                                                 
 random_flip (RandomFlip)    (None, 2, 2, 1024)        0         
                                                                 
 random_rotation (RandomRota  (None, 2, 2, 1024)       0         
 tion)                                                           
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 16)                65552     
                                                         

In [ ]:
# cnt = 0
# for layer in resnet.layers:
#   cnt = cnt+1
#   layer.trainable = False
#   if (cnt>=120):
#     break

In [ ]:
# model.summary()


In [ ]:
X_train = X_train.reshape(len(X_train),64,64,3)/255
X_test = X_test.reshape(len(X_test),64,64,3)/255
y_train = y_train.reshape(len(y_train),2)
y_test = y_test.reshape(len(y_test),2)

In [ ]:
X_train.shape , y_train.shape , X_test.shape , y_test.shape

((672, 64, 64, 3), (672, 2), (288, 64, 64, 3), (288, 2))

In [ ]:
opt = keras.optimizers.RMSprop(learning_rate=0.000001)
model.compile(opt, 'binary_crossentropy', ['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train, validation_data= (X_test, y_test), epochs= 1000)

Epoch 1/1000
21/21 [==============================] - 25s 870ms/step - loss: 1.0416 - accuracy: 0.4613 - val_loss: 0.7626 - val_accuracy: 0.5764
Epoch 2/1000
21/21 [==============================] - 17s 830ms/step - loss: 1.0761 - accuracy: 0.4762 - val_loss: 0.7608 - val_accuracy: 0.5451
Epoch 3/1000
21/21 [==============================] - 19s 887ms/step - loss: 1.0004 - accuracy: 0.5164 - val_loss: 0.7668 - val_accuracy: 0.5069
Epoch 4/1000
21/21 [==============================] - 17s 826ms/step - loss: 0.9590 - accuracy: 0.5074 - val_loss: 0.7608 - val_accuracy: 0.5104
Epoch 5/1000
21/21 [==============================] - 17s 827ms/step - loss: 0.9716 - accuracy: 0.4940 - val_loss: 0.7711 - val_accuracy: 0.5139
Epoch 6/1000
21/21 [==============================] - 17s 830ms/step - loss: 0.9735 - accuracy: 0.5179 - val_loss: 0.7847 - val_accuracy: 0.4896
Epoch 7/1000
21/21 [==============================] - 17s 826ms/step - loss: 0.9787 - accuracy: 0.4985 - val_loss: 0.7965 - val_ac

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
pred = model.predict(X_test)

In [ ]:
len(X_test)

In [ ]:
pred_li = [np.argmax(i) for i in pred]

In [ ]:
pred_li

In [ ]:
test_li = [np.argmax(i) for i in y_test]

In [ ]:
test_li

In [ ]:
from sklearn import metrics
# print(confusion_matrix(test_li, pred_li))
confusion_matrix = metrics.confusion_matrix(test_li, pred_li)


cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["Cat", "dog"])



In [ ]:
cm_display.plot()
plt.show()

In [ ]:
new_img = '/content/drive/MyDrive/ResNet Data/ResNet Data/dog02.jpg'
img_array = cv2.imread(new_img)
# img_array = cv2.cvtColor(img_array,cv2.COLOR_GRAY2RGB)
plt.imshow(img_array)

In [ ]:
new_array = cv2.resize(img_array, (64, 64))
print(new_array.shape)
test = np.expand_dims(new_array, axis = 0)
print(test.shape)
pred = model.predict(test)
print(f'Predictions : {pred}')

In [ ]:
print('Actual Input : ', new_img.split('/')[-1][:3].capitalize())
if np.argmax(pred) == 1:
  print('Model Predicted : Dog')
else:
  print('Model Predicted : Cat')

In [ ]:
path = '/content/drive/MyDrive/ResNet Data/ResNet Data/Cats'
path01 = '/content/drive/MyDrive/ResNet Data/ResNet Data/Dogs'

In [ ]:
#Counting Data
cat_li = []
for img in os.listdir(path):
    cat_li.append(img)

dog_li = []
for im in os.listdir(path01):
    dog_li.append(im)

In [ ]:
len(cat_li), len(dog_li)